**WARNING:** Meant to be a helpful reference and not complete documentation. May not be completely accurate, this is just my understanding of what's going on.\
See documentation [here!](https://github.com/holiday-pettijohn/Stylus_Scoring_Generalization/blob/main/Documentation/matching_api.md)

## Terminology

archetype: perfect Han character, original data from .han files\
bases: ATGC string\
gene: imperfect representation of Han character, data to be scored from .gene files\
reference character: unicode value for han character. *example: 4EFB*

## Function Reference

**exhaustScore:** Returns the maximum exhaustive score based on the reference character

**extractBases:** Returns loadGeometryBases output for one gene (so not all arrays) but excluding g_data and f_names

**loadGeometryBases:** Returns the values below for each gene (in order) based on the directory with genes. __Return values will all be arrays of the output described because it returns each value for every gene.__
- g_data: xmlToGeometry output (as a list)
    - g: geometric strokes
    - l: "progress percentage" for each geometric stroke
- han_chars: reference character of the gene (each gene file already has a reference character associated with it)
- base_data: bases of the gene
- stroke_sets: gene strokes
- stroke_orders: gene stroke order
- f_names: file name of the gene

**loadRef:** Returns a list of archetype strokes and the "progress percentages" for each stroke based on a reference character
- Process: Goes into the Reference directory > Gets archetype that matches reference character > Returns relevant data
- ref_g: list of strokes
- ref_l: fractional distances
- output_size: size for the bounding box (?)

**strokeErrorMatrix:** Returns a matrix of errors between every geometric stroke (columns) and archetype stroke (rows)

**getXmlScore:** Returns a score based on XML
  
**minXml:** Returns XML to give to Stylus API

**xmlToGeometry:** Returns a list of geometric strokes and the "progress percentages" for each stroke based on a gene file and a bounding box
- Process: Goes into gene file provided > Returns relevant data

## Strokes

Both archetype strokes and geometric strokes consist of location points on an (x, y) scale (incoherent vectors are filtered out). Gene strokes correspond to the position of the bases.

## Overall Process

__Exhaustive:__
1. Load Han archetype data (loadRef)
2. Load every gene's data from directory (loadGeometryBases)
3. Iterate over every gene in the directory
4. Scores the archetype/gene combo for every possible stroke order (minXml, getXmlScore)
5. Iterate over each gene's exhaustive scores and return maximum score

__Heuristic (Greedy):__
1. Load Han archetype data (loadRef)
2. Load every gene's data from directory (loadGeometryBases)
3. Iterate over every gene in the directory
4. Get error between archetype/gene strokes (strokeErrorMatrix)
5. Retrieve the lengths of each archetype stroke (getStrokeLen)
6. Iterate over every archetype stroke
7. Take the largest archetype stroke and access the row it corresponds to on the error matrix
8. Find the geometric stroke with the smallest error
9. Add the stroke to the stroke order *(example: the geometric stroke 4 had the smallest error when searching for archetype stroke 1, so stroke_map[4] = 1)*
10. Remove the largest archetype stroke from the list and make sure no geometric stroke is used twice 
11. Score the archetype/gene combo using the stroke order obtained through the heuristic algorithm (minXml, getXmlScore)

The exhaustive will test every single possible stroke order while the heuristic narrows it down to a single stroke order.